In [3]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import music21

In [4]:
model = tf.keras.models.load_model('trained_model.h5')

Beschränkung liegt hier bei einem Lied mit 8 Takten. Kann auch erweitert werden. Allerdings benötigt man dann ein neu traniertes Model. Das ist der Nachteil bei diesem Ansatz

In [5]:
given_melody = '''
A4   --   A4   --   |G#4  --   --   --   |E4  --   --   --   |F4   --   --   --   |
D4   --   --   --   |--   --   --   --   |D4   --   --   --   |--   --   --   --   |
E4   --   --   --   |F#4  --   --   --   |G#4  --   --   --   |A4   --   B4   --   |
C5   --   --   --   |C4   --   --   --   |E4   --   --   --   |--   --   --   --   |
A4   --   --   --   |G#4  --   --   --   |A4   --   --   --   |F4   --   --   --   |
D4   --   --   --   |--   --   --   --   |D4   --   --   --   |--   --   --   --   |
E4   --   --   --   |F#4  --   --   --   |G#4  --   --   --   |A4   --   B4   --   |
C5   --   --   --   |C4   --   --   --   |E4   --   --   --   |A4   --   --   --   |
'''

In [6]:
given_melody = given_melody.replace('\n', '').replace('|', '')

In [7]:
tokens = given_melody.split()
print(len(tokens))

128


In [8]:
SOPRANO_MIN = 57
SOPRANO_MAX = 81

ALTO_MIN = 52
ALTO_MAX = 74

TENOR_MIN = 48
TENOR_MAX = 69

BASS_MIN = 36
BASS_MAX = 64

ranges = {
    'soprano': {midinumber: (midinumber - SOPRANO_MIN + 1) for midinumber in range(SOPRANO_MIN, SOPRANO_MAX + 1)},
    'alto': {midinumber: (midinumber - ALTO_MIN + 1) for midinumber in range(ALTO_MIN, ALTO_MAX + 1)},
    'tenor': {midinumber: (midinumber - TENOR_MIN + 1) for midinumber in range(TENOR_MIN, TENOR_MAX + 1)},
    'bass': {midinumber: (midinumber - BASS_MIN + 1) for midinumber in range(BASS_MIN, BASS_MAX + 1)},
}

reverse_ranges = {
    'soprano': {(midinumber - SOPRANO_MIN + 1): midinumber for midinumber in range(SOPRANO_MIN, SOPRANO_MAX + 1)},
    'alto': {(midinumber - ALTO_MIN + 1): midinumber for midinumber in range(ALTO_MIN, ALTO_MAX + 1)},
    'tenor': {(midinumber - TENOR_MIN + 1): midinumber for midinumber in range(TENOR_MIN, TENOR_MAX + 1)},
    'bass': {(midinumber - BASS_MIN + 1): midinumber for midinumber in range(BASS_MIN, BASS_MAX + 1)},
}

In [9]:
def encode_note(n, rang):
    if n == '--' or n == 'Rest':
        ret = 0
    else:
        note = music21.note.Note(n)
        ret = ranges[rang][note.pitch.midi]
    return ret

def one_hot_encode(idx, rang):
    length = len(ranges[rang].values())
    ret = [0] * (length + 1)
    ret[idx] = 1
    return ret

In [10]:
s = [encode_note(n, 'soprano') for n in tokens] 
x = np.array([[one_hot_encode(idx, 'soprano') for idx in s]])
 # zu einem 1d array machen
x = x.reshape(1, -1)

The melody has been encoded, so we can pass it to the model and collect the predictions from the MiniBach model.

Jetzt unterteilt man die Predictions in die verschieden Stimmen mit folgender Rechnung. Wir wissen, dass wir eine Tonrange bei Alto von 23+1 haben. Wir wissen auch unsere Chunckgröße von 128. Also:

128*24=3072

182*23=2944
3072+2944=6016

...

In [11]:
predictions = model.predict(x)

predictions = predictions.reshape(-1)

print(len(predictions))

soprano = x.reshape(128, -1)
alto = predictions[:3072].reshape(128, -1)
tenor = predictions[3072:6016].reshape(128, -1)
bass = predictions[6016:9856].reshape(128, -1)


music = {
    'soprano': soprano,
    'alto': alto,
    'tenor': tenor,
    'bass': bass
}

1/1 [==============================] - 0s 133ms/step
9856


In [12]:
def decode_note(n, rang):
    if n == 0:
        ret = '--'
    else:
        note = music21.note.Note(type='16th')        
        note.pitch.midi = reverse_ranges[rang][n]        
        ret = note
    return ret

In [13]:
generation = {
    'soprano': [],
    'alto': [],
    'tenor': [],
    'bass': []
}

for sixteenth in range(128):
    for part, notes in music.items():
        this_note = decode_note(np.argmax(notes[sixteenth]), part)
        if this_note == '--':
            last_note = generation[part][-1]
            this_note = music21.note.Note(last_note.pitch.nameWithOctave, type='16th')
            if last_note.tie:
                this_note.tie = music21.tie.Tie('continue')
            else:
                last_note.tie = music21.tie.Tie('start')
                generation[part][-1] = last_note
                this_note.tie = music21.tie.Tie('continue')
        else:
            if sixteenth > 0:
                last_note = generation[part][-1]
                if last_note.tie:
                    last_note.tie = music21.tie.Tie('stop')
        generation[part].append(this_note)

In [14]:
df = pd.DataFrame(generation)

In [17]:
s = music21.stream.Stream()
s.append(df.soprano.to_list())
a = music21.stream.Stream()
a.append(df.alto.to_list())
print(df.alto.to_list())
t = music21.stream.Stream()
t.append(df.tenor.to_list())
b = music21.stream.Stream()
b.append(df.bass.to_list())
stream = music21.stream.Stream([s,a,t,b])

In [16]:
stream.write('musicxml', 'generated_choral.musicxml')
stream.write('midi', 'generated_choral.mid')

'generated_choral.mid'